In [1]:
import pandas as pd
import numpy as np

### Read in and Preprocess image data

In [20]:
listdf = pd.read_csv('annotations/list.txt',skiprows = 6, sep = ' ', 
                     header = None, names = ['Image','Class_ID','Species','Breed'])
listdf.head()

,Image,Class_ID,Species,Breed
0,Abyssinian_100,1,1,1
1,Abyssinian_101,1,1,1
2,Abyssinian_102,1,1,1
3,Abyssinian_103,1,1,1
4,Abyssinian_104,1,1,1


In [83]:
listdf.shape

(7349, 4)

In [106]:
from PIL import Image
from keras.preprocessing import image

labels = listdf['Class_ID'].values
image_names = listdf['Image']
X = []
for name in image_names:
    img = Image.open('images/'+name+'.jpg').resize((224,224))
    if img.mode != 'RGB':
            img = img.convert('RGB')
    x = image.img_to_array(img)
    X.append(x)

In [107]:
X = np.array(X)

### Feature extraction using MobileNet

In [109]:
#MobileNet
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
# load MobileNet
model = MobileNet(include_top=False, weights='imagenet', input_shape = (224,224,3))
# preprocessing for MobileNet
X_pre = preprocess_input(X)
features = model.predict(X_pre)
print(X.shape)
print(features.shape)
features_ = features.reshape(X.shape[0], -1)
Xdf = pd.DataFrame(data = features_)
Xdf.to_csv('features.csv',index = False)

(7349, 224, 224, 3)
(7349, 7, 7, 1024)


In [ ]:
#VGG16
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg16 import preprocess_input
# # load VGG16
# model = VGG16(include_top=False, weights='imagenet')
# # preprocessing for VGG16
# X_pre = preprocess_input(X)
# features = model.predict(X_pre)
# print(X.shape)
# print(features.shape)
# features_ = features.reshape(X.shape[0], -1)
# Xdf = pd.DataFrame(data = features_)
# Xdf.to_csv('features.csv',index = False)

#### After feature extraction with Mobilenet, we got a feature matrix with size (7349, 7, 7, 1024) before reshaping.

### Train LinearSVC and MLP models

In [112]:
X = pd.read_csv('features.csv').values
y = labels

In [121]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [124]:
from sklearn.svm import LinearSVC
svc = LinearSVC()
cv_svc = cross_val_score(svc, X_train, y_train, cv=5)
svc.fit(X_train, y_train)
print('cross_val_score: '+str(cv_svc.mean()))
print('test_score: '+str(svc.score(X_test, y_test)))

confusion_matrix(y_test, svc.predict(X_test))

cross_val_score: 0.881868017996
test_score: 0.882480957563


array([[46,  1,  0, ...,  0,  0,  0],
       [ 0, 40,  3, ...,  2,  0,  0],
       [ 0,  3, 34, ...,  6,  0,  0],
       ..., 
       [ 0,  3,  2, ..., 35,  0,  0],
       [ 0,  0,  0, ...,  0, 47,  0],
       [ 0,  0,  0, ...,  0,  1, 42]])

In [132]:
%%time
from sklearn.neural_network import MLPClassifier
cv_mlp = cross_val_score(MLPClassifier(), X_train, y_train, cv=3)
mlp = MLPClassifier()
mlp.fit(X_train, y_train)
print('cross_val_score: '+str(cv_mlp.mean()))
print('test_score: '+str(mlp.score(X_test, y_test)))

cm_mlp = confusion_matrix(y_test, mlp.predict(X_test))

cross_val_score: 0.700781866686
test_score: 0.763873775843
CPU times: user 17min 54s, sys: 4min 37s, total: 22min 32s
Wall time: 13min 31s


In [133]:
cm_mlp

array([[36,  1,  0, ...,  0,  0,  0],
       [ 0, 40,  7, ...,  0,  0,  0],
       [ 0,  8, 23, ...,  5,  0,  0],
       ..., 
       [ 0,  4,  3, ..., 34,  0,  0],
       [ 0,  0,  0, ...,  0, 42,  0],
       [ 0,  0,  0, ...,  0,  1, 37]])

#### For model selection, we tried two models LinearSVC and MLPClassifier with default parameters. And the LinearSVC model reached a cross_val_score and test score of 0.88. Meanwhile, the MLPClassifier got a cross_val_score of 0.70 and test score of 0.76. So we chose the LinearSVC model as the final model.